# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [ ]:
# Libraries
import numpy as np
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset admission predict from [here](https://drive.google.com/file/d/1y-8TOlbiSWUoMZ_Sg-WfOL2NySXP6i4e/view?usp=sharing]), place it in the data folder and load it

In [ ]:
# your code here
admissions= pd.read_csv('../data/Admission_Predict.csv')

Remove trailing spaces at the end of the column names if there are any.


In [ ]:
# your code here
for col in admissions.columns:
    col.strip()
    print(col) 

In [ ]:
admissions.columns

In [ ]:
collist = list(admissions.columns.values)
collist = [x.strip(' ') for x in collist]
print(collist) 
admissions.columns = collist

Let's evaluate the dataset by looking at the `head` function.

In [ ]:
# your code here
admissions.head()

Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [ ]:
# your code here
null_cols = admissions.isnull().sum()
null_cols
#no missing data in the data set

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [ ]:
# your code here
#admissions.append('Serial No. Index') = admissions.set_index('Serial No.')
#tried the above to duplicate the 'Serial No' column into an index, but it gave me the "can't assign to function call" error


# admissions['Serial No. Index'] = admissions['Serial No.']
# admissions.set_index('Serial No. Index')
# admissions.head()
# tried the above, but that requires adding another column

admissions.set_index(["Serial No."], inplace = True, 
                    append = False, drop = False) 
   
admissions.head()

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [ ]:
# your code here

print(admissions.set_index(['GRE Score']).index.is_unique)
# prints False
print(admissions.set_index(['CGPA']).index.is_unique)
# prints False
print(admissions.set_index(['GRE Score', 'CGPA']).index.is_unique)
# but this prints True

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [ ]:
# your code here
admissions.set_index(['GRE Score', 'CGPA'], inplace = True, 
                    append = False, drop = True) 
admissions.head()

In [ ]:
admissions.reset_index(inplace = True, drop = False)

admissions.head()

# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [ ]:
# your code here
CGPA9 = admissions.loc[(admissions['CGPA'] > 9) & (admissions['Research'] == 1)]
len(CGPA9)

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [ ]:
# your code here
CGPAless = admissions.loc[(admissions['CGPA'] > 9) & (admissions['SOP'] <= 3.5)]
CGPAless

In [ ]:
# Find the mean chance of admit for these applicants
CGPAdf = pd.DataFrame(data=CGPAless) 
CGPAdf['Chance of Admit'].mean()

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [ ]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    return [(elem - np.mean(col))/np.std(col) for elem in col]

#     return (col-col.mean())/col.std()

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [ ]:
# your code here
# admissions=data
# admissions[['CGPA_std', 'GRE Score_std', 'LOR_std']] = admissions[['CGPA', 'GRE Score', 'LOR'].apply(standardize)
#gives unexpected EOF while parsing
#tried:
#admissions.apply(pd.to_numeric, errors='ignore')
# it worked, but was still giving me error of int not being iterable
# admissions = admissions.astype(int)
# gave me parsing error, but if it worked, the result would still be int --> need list
# admissions['CGPA_std'] = admissions['CGPA'].apply(standardize)
# admissions['GRE_std'] = admissions['GRE Score'].apply(standardize)
# admissions['LOR_std'] = admissions['LOR'].apply(standardize)
# error: float object not iterable
GRE1 = admissions['GRE Score'].tolist()

CGPA1 = admissions['CGPA'].tolist()


CGPA_std = standardize(CGPA1)
admissions['CGPA_std'] = CGPA_std

GRE_std = standardize(GRE1)
admissions['GRE_std'] = GRE_std

LOR1 = admissions['LOR'].tolist()
LOR_std = standardize(LOR1)
admissions['LOR_std'] = LOR_std   


admissions.head()

We will generate the decision choice at random using the code below. Please run the cell.

In [ ]:
# Libraries
from random import choices

In [ ]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [ ]:
# your code here
admissions['decision_choice'] = decision_choice
admissions['deciding_column'] = admissions.lookup(admissions.index, admissions['decision_choice'])

admissions

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [ ]:
# your code here
admissions['decision'] = [1 if elem > 0.8 else 0 for elem in admissions['deciding_column']]
admissions.head()

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [ ]:
# your code here
admitted_list1 = admissions['decision'].tolist()
admitted_list2 = [ x for x in admitted_list1 if x == 1]
print (len(admitted_list2))

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
# your code here
admissions.columns = [x.lower() for x in admissions.columns]
admissions.columns = admissions.columns.str.replace(' ', '_')
admissions.columns = admissions.columns.str.replace('.', '' )
admissions.columns

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
# your code here
adjusted_gre1 = admissions.loc[admissions['university_rating'] >= 4, 'adjusted_gre'] = admissions['gre_score'] + 10
adjusted_gre2 = pd.cut(adjusted_gre1, 4, retbins=True)
admissions.head()